# Setup

In [ ]:
import numpy as np
import gc
import torch
from keras.preprocessing import image
import os
from tqdm import tqdm

os.chdir('../rfiw2021')

from Track1.models import Net
from Track1.utils import *


Defining constants

In [2]:
FILE = 'Track1'
SAMPLE = 'Track1/sample0/' # Path to sample image
MODEL_PATH = '../notebooks/data/model_track1.pth'
THRESHOLD = 0.11546290665864944 # Classification threshold
BATCH_SIZE = 40

Creating res object to keep the accuracies

In [3]:
classes = ['bb', 'ss', 'sibs', 'fd', 'md', 'fs', 'ms', 'gfgd', 'gmgd', 'gfgs', 'gmgs', 'avg']
res = {}
for n in classes:
    res[n]=[0,0]

Loading the model

In [ ]:
model = Net().cuda()
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

Loading the test database

In [5]:
test_file_path = os.path.join(SAMPLE,"test.txt")
test_samples=[]
f = open(test_file_path, "r+", encoding='utf-8')
while True:
    line=f.readline().replace('\n','')
    if not line:
        break
    else:
        test_samples.append(line.split(' '))
f.close()
res['avg'][0]=len(test_samples)
for now in test_samples:
    res[now[3]][0]+=1

# Training the model

In [6]:
def read_image(path):
    img = image.load_img(f"{FILE}/{path}", target_size=(112, 112))
    img = np.array(img).astype(float)
    return np.transpose(img, (2, 0, 1))

def gen(list_tuples, batch_size):
    total=len(list_tuples)
    start=0
    while True:
        if start+batch_size<total:
            end=start+batch_size
        else:
            end=total
        batch_list=list_tuples[start:end]
        datas=[]
        labels=[]
        classes=[]
        for now in batch_list:
            datas.append([now[1],now[2]])
            labels.append(int(now[4]))
            classes.append(now[3])
        X1 = np.array([read_image(x[0]) for x in datas])
        X2 = np.array([read_image(x[1]) for x in datas])
        yield X1, X2, labels,classes,batch_list
        start=end
        if start == total:
            yield None,None,None,None,None
        gc.collect()

In [7]:
with torch.no_grad():
    for img1, img2, labels, classes, batch_list in tqdm(gen(test_samples, BATCH_SIZE), total=len(test_samples) // BATCH_SIZE):
        if img1 is not None:
            img1 = torch.from_numpy(img1).type(torch.float).cuda()
            img2 = torch.from_numpy(img2).type(torch.float).cuda()
            em1, em2, _, _ = model([img1, img2])
            pred = torch.cosine_similarity(em1, em2, dim=1).cpu().detach().numpy().tolist()
            for i in range(len(pred)):
                if pred[i] >= THRESHOLD:
                    p = 1
                else:
                    p = 0
                if p == labels[i]:
                    res['avg'][1] += 1
                    res[classes[i]][1] += 1
        else:
            break

994it [08:31,  1.94it/s]                         


# Results

Saving the results in a log file

In [9]:
for key in res:
    mylog(key, ':', res[key][1] / res[key][0], path='testando')

bb : 0.8107118644067797
ss : 0.8119018618409605
sibs : 0.786174575278266
fd : 0.7393098893841836
md : 0.7911679068262698
fs : 0.8233587786259542
ms : 0.7614752455401884
gfgd : 0.7742663656884876
gmgd : 0.7509293680297398
gfgs : 0.7224489795918367
gmgs : 0.6033519553072626
avg : 0.7895226832398158
